# Lab 6-2: Fancy Softmax Classification

Author: Seungjae Lee (이승재)

## Imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# For reproducibility
torch.manual_seed(1)

## Cross-entropy Loss with `torch.nn.functional`

PyTorch has `F.log_softmax()` function.

In [ ]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
y = torch.randint(5, (3,)).long()
print(y)
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([1, 0, 0])


tensor([[0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [ ]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.4101, -1.9457, -1.4703, -1.4507, -1.9056],
        [-1.7925, -1.3848, -1.4657, -1.7679, -1.7066],
        [-1.2971, -1.8475, -1.4731, -1.5385, -2.0782]], grad_fn=<LogBackward0>)

In [ ]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.4101, -1.9457, -1.4703, -1.4507, -1.9056],
        [-1.7925, -1.3848, -1.4657, -1.7679, -1.7066],
        [-1.2971, -1.8475, -1.4731, -1.5385, -2.0782]],
       grad_fn=<LogSoftmaxBackward0>)

PyTorch also has `F.nll_loss()` function that computes the negative loss likelihood.

In [ ]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.6784, grad_fn=<MeanBackward0>)

In [ ]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y.long())

tensor(1.6784, grad_fn=<NllLossBackward0>)

PyTorch also has `F.cross_entropy` that combines `F.log_softmax()` and `F.nll_loss()`.

In [22]:
F.cross_entropy(z, y)

tensor(1.6784, grad_fn=<NllLossBackward0>)

## Data

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [23]:
xy = np.loadtxt('/content/drive/MyDrive/HYPER/data-04-zoo.csv', delimiter=',', dtype=np.float32)

In [31]:
x_train = torch.FloatTensor(xy[:, 0:-1])
y_train = torch.LongTensor(xy[:, [-1]]).squeeze()

In [32]:
print(x_train.shape) # x_train shape
print(len(x_train))  # x_train 길이
print(x_train[:5])   # 첫 다섯 개

torch.Size([101, 16])
101
tensor([[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 4., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 1., 0., 1.]])


In [33]:
print(y_train.shape) # y_train shape
print(len(y_train))  # y_train 길이
print(y_train[:5])   # 첫 다섯 개

torch.Size([101])
101
tensor([0, 0, 3, 0, 0])


In [34]:
nb_classes = 7 # csv 파일을 보시면 0~6까지 총 7개의 class인 것을 확인할 수 있습니다!
y_one_hot = torch.zeros((len(y_train), nb_classes))
y_one_hot = y_one_hot.scatter(1, y_train.unsqueeze(1), 1)

## Training with `F.cross_entropy`

In [36]:
# 모델 초기화
W = torch.zeros([16,7],requires_grad= True)#이 부분을 채워넣으세요.# #가중치 W를 0으로 초기화하고 학습을 통해 값이 변경될 수 있도록 하세요.
b = torch.zeros([1],requires_grad= True)#이 부분을 채워넣으세요.# #편향 b를 0으로 초기화하고 학습을 통해 값이 변경될 수 있도록 하세요.
# optimizer 설정
optimizer = optim.SGD([W,b],lr = 0.1)#이 부분을 채워넣으세요.# #SGD optimizer를 사용하고 learning rate는 0.1로 적용하세요.

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z,y_one_hot)#이 부분을 채워넣으세요.# #PyTorch에서 제공하는 F.cross_entropy함수를 사용해 cost를 구현해보세요.

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    #이 부분을 채워넣으세요.#  # gradient를 0으로 초기화
    #이 부분을 채워넣으세요.#  # 비용 함수를 미분하여 gradient 계산
    #이 부분을 채워넣으세요.#  # W와 b를 업데이트

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.945910
Epoch  100/1000 Cost: 0.471836
Epoch  200/1000 Cost: 0.326327
Epoch  300/1000 Cost: 0.257839
Epoch  400/1000 Cost: 0.215762
Epoch  500/1000 Cost: 0.186603
Epoch  600/1000 Cost: 0.164898
Epoch  700/1000 Cost: 0.147955
Epoch  800/1000 Cost: 0.134278
Epoch  900/1000 Cost: 0.122962
Epoch 1000/1000 Cost: 0.113422


## High-level Implementation with `nn.Module`

In [42]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(16,7) #x_train의 shape과 y_train의 class의 갯수를 고려해보세요.
    def forward(self, x):
        return self.linear(x)

In [43]:
model = SoftmaxClassifierModel()#이 부분을 채워넣으세요.# #SoftmaxClassifierModel()로 모델을 초기화하세요.

In [45]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(),lr = 0.1)#이 부분을 채워넣으세요.# #SGD optimizer를 사용하고 learning rate는 0.1로 적용하세요.

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)#이 부분을 채워넣으세요.# #모델을 생성해서 예측값을 구해보세요.

    # cost 계산
    cost = F.cross_entropy(prediction,y_one_hot)#이 부분을 채워넣으세요.# #PyTorch에서 제공하는 F.cross_entropy함수를 사용해 cost를 구현해보세요.

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    #이 부분을 채워넣으세요.#  # gradient를 0으로 초기화
    #이 부분을 채워넣으세요.#  # 비용 함수를 미분하여 gradient 계산
    #이 부분을 채워넣으세요.#  # W와 b를 업데이트
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.100695
Epoch  100/1000 Cost: 0.092905
Epoch  200/1000 Cost: 0.086239
Epoch  300/1000 Cost: 0.080468
Epoch  400/1000 Cost: 0.075423
Epoch  500/1000 Cost: 0.070976
Epoch  600/1000 Cost: 0.067025
Epoch  700/1000 Cost: 0.063493
Epoch  800/1000 Cost: 0.060317
Epoch  900/1000 Cost: 0.057444
Epoch 1000/1000 Cost: 0.054834


<div class="alert alert-warning">
    Should I display how many it got correct in the training set?
</div>

Q1. 각 Regression에 맞는 Hypothesis와 cost function을 적어주세요.

Linear Regression : H(x) = XW + b
                    loss  = (H - Y)**2

Logistic Regression : H = XW + b
                      S = 1/(1 + e**H)
                      loss = -y*log(1-S) - (1-y)*log(S)

Multi Logistic regression : H = XW + b
                            z = softmax(H)
                            loss = -y*log(z)

Q2. Lec06-2의 Logistic cost VS Cross entropy에서 두 함수가 같은 이유를 서술해주세요.

=> 모르겠다